In [1]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 11.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=2811640 sha256=34e12f7f1ea31c0d1d3ab0297c80ed4d61546c84005f35838e8682e344331f2c
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [2]:
import pandas as pd
import numpy as np
import surprise
import os

In [3]:
os.chdir("/content/drive/MyDrive/Cases/filmtrust")
ratings = pd.read_csv("ratings.txt",sep=' ',names = ['uid','iid','rating'])
ratings.head()

,uid,iid,rating
0,1,1,2.0
1,1,2,4.0
2,1,3,3.5
3,1,4,3.0
4,1,5,4.0


In [4]:
lowest_rating = ratings['rating'].min()
highest_rating = ratings['rating'].max()
print("Ratings range between {0} and {1}".format(lowest_rating,highest_rating))

Ratings range between 0.5 and 4.0


In [5]:
reader = surprise.Reader(rating_scale = (lowest_rating,highest_rating))
data = surprise.Dataset.load_from_df(ratings,reader)
type(data)

surprise.dataset.DatasetAutoFolds

In [6]:
similarity_options = {'name': 'cosine', 'user_based': False}
# Default k = 40
algo = surprise.KNNBasic(sim_options = similarity_options)
output = algo.fit(data.build_full_trainset())

Computing the cosine similarity matrix...
Done computing similarity matrix.


Expected rating for user 50 for item 217:

In [7]:
pred = algo.predict(uid='50',iid='217')
print(pred.est)

3.0028030537791928


Total Items:

In [8]:
iids = ratings['iid'].unique()
print(iids)

[   1    2    3 ... 2069 2070 2071]


The list of items rated by user 50:

In [9]:
u_iid = ratings[ratings['uid']==50]['iid'].unique()
print(u_iid)

[  8 211   3   2 219 234  12 254 250 207  11 253 236  84  10   7 233  13
   1   5   6 252 241 216 257 206   4 217   9 215 213  17 255 220 121 245
 239 251 235]


In [10]:
len(u_iid)

39

List of the items not rated by user 50:

In [11]:
iids_to_predict = np.setdiff1d(iids, u_iid)
print(iids_to_predict)

[  14   15   16 ... 2069 2070 2071]


In [12]:
len(iids_to_predict)

2032

Extracting the estimated rating from iids_to_predict

In [ ]:
testset = [[50,iid,0.] for iid in iids_to_predict]
testset

In [ ]:
testset = [[50,iid,0.] for iid in iids_to_predict]
predictions = algo.test(testset)

In [ ]:
type(predictions)

list

In [ ]:
predictions[:5]

[Prediction(uid=50, iid=14, r_ui=0.0, est=3.607142857142857, details={'actual_k': 14, 'was_impossible': False}),
 Prediction(uid=50, iid=15, r_ui=0.0, est=3.5269249184397617, details={'actual_k': 39, 'was_impossible': False}),
 Prediction(uid=50, iid=16, r_ui=0.0, est=3.565403846185209, details={'actual_k': 23, 'was_impossible': False}),
 Prediction(uid=50, iid=18, r_ui=0.0, est=3.607142857142857, details={'actual_k': 14, 'was_impossible': False}),
 Prediction(uid=50, iid=19, r_ui=0.0, est=3.607142857142857, details={'actual_k': 14, 'was_impossible': False})]

In [ ]:
len(predictions)

2032

In [ ]:
predictions[4].est

2.9501774353655654

In [ ]:
pred_ratings = np.array([pred.est for pred in predictions])
pred_ratings

array([3.60714286, 3.52692492, 3.56540385, ..., 3.75      , 3.75      ,
       3.46875   ])

In [ ]:
np.argmax(pred_ratings)

20

In [ ]:
iids_to_predict[20]

35

In [ ]:
print(iids_to_predict)

[  14   15   16 ... 2069 2070 2071]


Getting the item with highest expected rating

In [ ]:
i_max = np.argmax(pred_ratings)
print("Item:",iids_to_predict[i_max],"is the item with highest expected rating as",pred_ratings[i_max])

Item: 258 is the item with highest expected rating as 4.0


In [ ]:
exp_ratings = pd.DataFrame({'Item_ID': iids_to_predict, 'Exp_Rating':pred_ratings})
exp_ratings.head()

,Item_ID,Exp_Rating
0,14,3.607143
1,15,3.526925
2,16,3.565404
3,18,3.607143
4,19,3.607143


In [ ]:
sorted_exp = exp_ratings.sort_values(by=['Exp_Rating','Item_ID'], ascending=[False, True])
sorted_exp.head(10)

,Item_ID,Exp_Rating
218,258,4.000000
1766,1806,4.000000
1805,1845,4.000000
1167,1207,3.833333
1165,1205,3.785714
896,936,3.750000
1470,1510,3.750000
1595,1635,3.750000
1647,1687,3.750000
1689,1729,3.750000


Tuning for best K

In [ ]:
from surprise.model_selection import GridSearchCV
from surprise.model_selection.split import KFold

In [ ]:
param_grid = {'k': np.arange(30,70,10),
               'sim_options': {'name': ['cosine'],
                              'user_based': [False]}}
param_grid

{'k': array([30, 40, 50, 60]),
 'sim_options': {'name': ['cosine'], 'user_based': [False]}}

In [ ]:
kfold = KFold(n_splits=5, random_state=2023, shuffle=True)
gs = GridSearchCV(surprise.KNNBasic, param_grid,
                  measures=['rmse', 'mae'], cv=kfold)

In [ ]:
gs.fit(data)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing th

Best Score:

In [ ]:
print(gs.best_score['rmse'])

0.8248813318569027


Best Parameter:

In [ ]:
print(gs.best_params['rmse'])

{'k': 30, 'sim_options': {'name': 'cosine', 'user_based': False}}


We can now use the algorithm that yields the best rmse:

In [ ]:
algo = gs.best_estimator['rmse']
algo.fit(data.build_full_trainset())

Computing the cosine similarity matrix...
Done computing similarity matrix.


The list of items rated by user 50:

In [ ]:
u_iid = ratings[ratings['uid']==50]['iid'].unique()
print(u_iid)

[  8 211   3   2 219 234  12 254 250 207  11 253 236  84  10   7 233  13
   1   5   6 252 241 216 257 206   4 217   9 215 213  17 255 220 121 245
 239 251 235]


List of the items not rated by user 50:

In [ ]:
iids_to_predict = np.setdiff1d(iids, u_iid)
print(iids_to_predict)

[  14   15   16 ... 2069 2070 2071]


Extracting the estimated rating from iids_to_predict

In [ ]:
testset = [[50,iid,0.] for iid in iids_to_predict]
predictions = algo.test(testset)

In [ ]:
predictions[0]

Prediction(uid=50, iid=14, r_ui=0.0, est=3.607142857142857, details={'actual_k': 14, 'was_impossible': False})

In [ ]:
pred_ratings = np.array([pred.est for pred in predictions])

Getting the item with highest expected rating

In [ ]:
i_max = np.argmax(pred_ratings)
print("Item:",iids_to_predict[i_max],"is the item with highest expected rating as",pred_ratings[i_max])

Item: 258 is the item with highest expected rating as 4.0


In [ ]:
exp_ratings = pd.DataFrame({'Item_ID': iids_to_predict, 'Exp_Rating':pred_ratings})
exp_ratings.head()

,Item_ID,Exp_Rating
0,14,3.607143
1,15,3.534078
2,16,3.565404
3,18,3.607143
4,19,3.607143


In [ ]:
sorted_exp = exp_ratings.sort_values(by=['Exp_Rating','Item_ID'], ascending=[False, True])
sorted_exp.head(10)

,Item_ID,Exp_Rating
218,258,4.000000
1766,1806,4.000000
1805,1845,4.000000
1167,1207,3.833333
1165,1205,3.785714
896,936,3.750000
1470,1510,3.750000
1595,1635,3.750000
1647,1687,3.750000
1689,1729,3.750000
